<!--Document and model descriptions-->

*This document steps through the process of linking the (physical)* [Kaneohe, Oahu ROMS model](https://www.pacioos.hawaii.edu/metadata/roms_hioekg_forecast.html?format=fgdc) *and the (biogeochemical)* [COBALT model](https://www.sciencedirect.com/science/article/pii/S0079661113001079).

*This is a live document that will be updated as needed.*

*Last update:* 2019-01-31

# How-to: linking the physical and biogeochemical components of COBALT

## Setting up the physical (ROMS) model

1. **roms_config.sh**
  * Change ``ROMS_SRC=[current dir]``.
  * Ensure ``GIT_BRANCH=river``. 
  
  
2. **hioekg.h**
  * At this stage (getting only the physical model to run), no changes.
  
  
3. ``rmake -j2``
  * This creates ``/build`` directory. If I want to change the source code, I can do that in ``/build``, then go to the main directory and type ``rmake -j2`` to recompile.
  * **oceanM*** is the executable file that is created.
  * **varinfo.dat** is also created.
  
  
4. **ocean.in**
  * If changed, I *don’t* need to recompile using ``rmake -j2``.
  * In this file, I tell the model where to find the starting NetCDF files:
    1. ``GRDNAME``: Change pathway to **hioekg-grid.nc**.
    2. ``ININAME``: Change pathway to **his-ini-##.nc**. Note that the initialization file must contain the keyword “his”, or the script won’t be able to find it- this is an inherent ROMS oddity.
    3. ``BRYNAME``: Change pathway to **hioekg-bry.nc**, the boundary conditions file.
    4. ``CLMNAME``: Change pathway to **hioekg-clim-##.nc**, the climatology file.
    5. ``SSFNAME``: Change pathway to **river-hioekg.nc**, the sources/sinks file.
    6. ``TIDENAME``: Change pathway to **hioekg-tide.nc**, the tidal forcing file.
    7. ``FRCNAME``: Change pathway to **frc-wrfo.nc**, the forcing file. Note that I can change ``NFFILES = #``, located a few lines above, to point to multiple forcing files. This will happen later, when COBALT is run.
    8. ``DSTART``: Change to match the ## on **his-ini-##.nc**. Presently, **day2date.sh** and **date2day.sh** are in ``/nest``. These scripts translate dates and days. Example: ``date2day.sh 2011 1 1 → ##`` (translates to a day).
    9. ``TIDESTART``: Change to match ## sourced from ``ncdump -h ./ncfiles/hioekg-tide.nc``.
    10. ``NTIMES = #`` of timesteps = # of seconds in your timeframe (= 86,400 * # days)/DT (seconds per step).
    11. ``NHIS``: How often output is written in # seconds, e.g., 360 = every hour).
    12. ``NtileI, NtileJ``: Set at 8 and 8. Related to the number of cores used to run the model.
    
    
5. **spinup.sh**
  * More powerful than (can overwrite) **ocean.in**.
    1. ``ROMS_START_DAY``: Change to match the ## on **his-ini-##.nc**.
    2. ``ROMS_END_DAY``: Change to end day (n = number of run days; end day = start + n-1. Example: start day = 4018, end day = 4020, n = 3).
    3. ``ROMS_RUN_DAYS``: Change to n.
    4. ``ROMS_QUEUE_NAME``: Change to default.
    5. ``ROMS_QUEUE``: Change to match ``NtileI, NtileJ``.
    6. ``ROMS_RUN_NAME``: Ensure there are no spaces!
    7. ``ROMS_FAIL_INPUT``: In case of failure, I can name a backup file. For now, left empty.

## Running ROMS

1. Copy the **his-ini-##.nc** file to ``/output``. ``cycle_roms`` will look for it there.


2. Dry run: 
```python
cycle_roms -d spinup.sh
``` 
...in ``/work``, this command creates…
    * **ini.nc**: restart file, copied by the model from ``/output``. The model will automatically change the dates, etc. according to specified cycle conditions.
    * **irp.nc**: ignore.
    * **ocean.in**: copied here, with automatically changed dates according to our pre-specified conditions.
    * **varinfo.dat**
    
    
3. If no dry run errors: 
```python
cycle_roms spinup.sh
```
...then...
```python
qstat
```
    * Did it crash? Check run log in ``/work``
    * If it crashed:
```python
    rm -rf ./work/his-ini*.nc HIOEKG_TESTING*
```
    * Be sure to replace ini file in /output


## Setting up COBALT

1. Navigate to ``/roms``. Then:
```python
git pull
git checkout origin/river -b river
```


2. Navigate to the main directory. Then:
```python
rmake clean
# Waiting...
rmake -j2```
...As with the ROMS protocol, this creates ``/build``, **oceanM***, and **varinfo.dat**.


3. Copy and rename **.in** file from another iteration. Example: 
```python
cp /share/server/person/directory/another_cobalt.in /share/server/me/mydirectory/bio_cobalt.in
```


4. **bio_cobalt.in**:
    + ``!Hout(idTvar) == 33*T`` ….this tells the model to write all 33 variables. Comment out (!) for now.
    * Checked **ocean.in** and found that boundary conditions vary for the hioekg grid. They are all uniform for hiig. We changed cols 1, 3, 4 to ``Cla `` and col 2 to ``Clo``.
    
    
5. **hioekg.h:
    * Copy the following segment from another **.h** file iteration and ensure ``DIAGNOSTICS_BIO`` and ``DIAGNOSTICS`` are *undefined*:
```python
#define BIOLOGY
#define BIO_COBALT
#if defined BIO_COBALT
# define OPTIC_MANIZZA
# define COBALT_MINERALS
# define COBALT_PHOSPHORUS
# define COBALT_IRON
# define COBALT_CARBON
# undef DIAGNOSTICS        /* diagnostics files contain Chl-a and .. */
# undef DIAGNOSTICS_BIO   /* derived biol. val. needed for restart and analysis
# undef ANA_BIOLOGY        /* analytical biology initial conditions */
# define ANA_BPFLUX        /* analytical bottom passive tracers fluxes */
# define ANA_SPFLUX        /* analytical surface passive tracers fluxes */
#endif
```


6. Navigate to main directory. Then: 
```python
rmake -j2
```


7. **ocean.in**
  * Change all ``Dout(*) == T`` to ``== F``. This saves storage and time when ``DIAGNOSTICS`` (which applies to ROMS and COBALT) is defined.
  * The structure of this file needs to be changed slightly. Manually compare another iteration of **ocean.in** (that has already been modified for COBALT usage).
  * ``LtracerSrc == F T``
  * Comment out ``TIDENAME ==``. Instead, put pathway to tide file under ``FRCNAME ==``.
  * Also add **frc-cobalt.nc** and **frc-wrfo.nc** under ``FRCNAME ==``.
  * Change the number of files specified by ``NFFILES`` accordingly.
  * ``BPARNAM =`` pathway to **hioekg_cobalt.in**.
  * Go to the ``BRYNAME`` section. Define ``NCLMFILES`` and ``NBCFILES`` as in the example file you are mimicking. 
  
  
8. Change specifications in **spinup.sh** as done when only running ROMS.

## Create NetCDF files to run COBALT

1. Navigate to ``/nest``.


2. **setup_inner_frc.py**
    * Chunk ``#1``: Sets up forcing from rivers, atmosphere; loads from internet. Note that if I run this and create a forcing file, but then need to change something, I need to delete that previously created file. otherwise, the script will “see” that file exists and not recreate it.
    * Chunk ``#2``: Creates **frc-cobalt.nc**.
    * Chunk ``#3``: Creates **river-hioekg.nc**. It may be helpful to look at documentation of ``psource.create``.
            i. I can change what I specify under ``seapy.roms.cobalt.add_psource_var(…``.
            ii. Temperature doesn’t really matter.
            iii. We always want salinity to = 5; it's a good approximation right now.
    * To run:
```python
qexec python setup_inner_frc.py
```


3. **setup_inner_nest.py**
    * Here, change only the start day, end day, and pathway (maybe) to outer grid.
    * To run:
```python
qexec -P4 python setup_inner_nest.py
```

## Running COBALT 

To run COBALT, use the same protocol as with ROMS. It's always helpful to do a dry run first to ensure everything is in order.